In [1]:
import numpy as np
from transformers import AutoModelForTokenClassification,AutoTokenizer
from transformers import pipeline
import time
from collections import Counter
import re

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the model and build the pipeline

In [2]:
model = AutoModelForTokenClassification.from_pretrained('/workspace/Distilbert_NER').to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained('/workspace/Distilbert_NER')

In [3]:
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple",device = 0) # pass device=0 if using gpu

# Spell correction and Classification

In [6]:
vocab = [['>','>=', 'more', 'beyond', 'over', 'costlier', 'higher', 'above', 'greater','atleast', 'at least', 'minimum', 'min'],
         ['<','<=','less', 'within', 'under', 'lower ', 'cheaper', 'below', 'lesser','at most', 'atmost', 'maximum', 'max'],
         ['=', 'equivalent', 'parallel', 'equal', 'similar', 'akin', 'comparable', 'for']]

In [7]:
f = open('/workspace/big.txt','w+')
for i in vocab:
    for j in i:
        f.write(j)
        f.write(' ')
f.write('than to')

7

In [12]:
def words(text): \
    return re.findall(r'\w+', text.lower())

def read():
    f.seek(0)
    return f.read()

WORDS = Counter(words(read()))
WORDLIST = list(WORDS.keys())

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or ['NA'])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Inference 

In [5]:
text = input()
start = time.time()

a = pipe(text)

end = time.time()
execution_time_milliseconds = (end-start) * 1000
print(f"Execution time: {execution_time_milliseconds:.2f} milliseconds")
for i in range(len(a)):
    print(f"{a[i]['entity_group']} : {a[i]['word']}")

 shirts under 500


Execution time: 24.75 milliseconds
comparison : under
price : 500


In [13]:
comp = ''
pr = ''
text = input()
start = time.time()
output = pipe(text)

for j in range(len(output)):
    if(output[j]['entity_group']=='comparison'):
        comp += output[j]['word'] + ' '
    if(output[j]['entity_group']=='price'):
        pr += output[j]['word'] + ' '
comp = comp.replace('than','').replace('to','').strip()
comp = ' '.join([correction(i) for i in comp.split()])
term = comp
comp = comp.split()
occurrence_lists = np.array([[[k in i for i in vocab[i]].count(True) for i in range(3)] for k in comp])
pos = -1 if (np.sum(occurrence_lists,axis=0)==[0,0,0]).all() else np.argmax(np.sum(occurrence_lists,axis=0))

end = time.time()
execution_time_milliseconds = (end-start) * 1000
comp = '>' if pos==0 else '<' if pos==1 else '='
#print(f"Execution time: {execution_time_milliseconds:.2f} milliseconds")
print(f"comparison : \t{term}")
print(f"price : \t{pr}")
print(f"comp: \t\t{comp}")


 blue shirts under 200


comparison : 	under
price : 	200 
comp: 		<
